In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [2]:
ratings = pd.read_csv('./data/input/ratings.csv')
movies = pd.read_csv('./data/input/movies.csv')

In [30]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
item_list = []
user_list = []
user_item_rating = defaultdict(dict)
for i in range(ratings.shape[0]):
    user = ratings.iloc[i]['userId']
    item = ratings.iloc[i]['movieId']
    if not item in item_list:
        item_list.append(item)
    if not user in user_list:
        user_list.append(user)
    user_item_rating[user][item] = ratings.iloc[i]['rating']

In [23]:
item_list.sort()
user_item_array = np.zeros((len(user_list), len(item_list)))
for user in user_list:
    for item, rating in user_item_rating[user].items():
        user_item_array[int(user - 1), int(item_list.index(item))] = rating

In [27]:
user_item_array

array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [32]:
comment = """


item_user = defaultdict(set)
items_set = set()
user_item_rating = defaultdict(dict)
for i in range(ratings.shape[0]):
    user = ratings.iloc[i]['userId']
    item = ratings.iloc[i]['movieId']
    items_set.add(item)
    item_user[item].add(user)
    user_item_rating[user][item] = ratings.iloc[i]['rating']

def sim(i, j):
    user_set = item_user[i].intersection(item_user[j])
    num = 0
    denom_1 = 0
    denom_2 = 0
    for user in user_set:
        ratings = user_item_rating[np.where(user_item_rating user)]
        mean_r = np.mean(list(ratings.values()))
        num += (ratings[i] - mean_r) * (ratings[j] - mean_r)
        denom_1 += (ratings[i] - mean_r) ** 2
        denom_2 += (ratings[j] - mean_r) ** 2
    if (denom_1 + denom_2) == 0:
        return 0
    return num / np.sqrt(denom_1 * denom_2)



"""

In [8]:
def sim(i, j):
    user_set = item_user[i].intersection(item_user[j])
    num = 0
    denom_1 = 0
    denom_2 = 0
    for user in user_set:
        ratings = user_item_rating[np.where(user_item_rating user)]
        mean_r = np.mean(list(ratings.values()))
        num += (ratings[i] - mean_r) * (ratings[j] - mean_r)
        denom_1 += (ratings[i] - mean_r) ** 2
        denom_2 += (ratings[j] - mean_r) ** 2
    if (denom_1 + denom_2) == 0:
        return 0
    return num / np.sqrt(denom_1 * denom_2)

# New try

In [5]:
import sys
import numpy as np

sys.stdin = open('./data/output/mapper_1.csv')
sys.stdout = open('./data/output/reducer_1.csv', 'w+')
line = sys.stdin.readline().rstrip('\n')
UserID, values = line.split('\t')
item, rating = values.split(':')
ItemID = [item]
Rating = [float(rating)]

for line in sys.stdin:
    line = line.strip()
    key, values = line.split('\t')
    if key == UserID:
        item, rating = values.split(':')
        ItemID.append(item)
        Rating.append(float(rating))
    else:
        ave = np.mean(Rating)
        Rating = Rating - ave
        length = len(ItemID)
        for i in range(length):
            DR_i = str(Rating[i])
            Item_i = ItemID[i]
            for j in range(i + 1, length):
                print('%s:%s\t%s:%s' % (Item_i, ItemID[j], DR_i, str(Rating[j])))
        UserID = key
        item, rating = values.split(':')
        ItemID = [item]
        Rating = [float(rating)]
ave = np.mean(Rating)
Rating = Rating - ave
length = len(ItemID)
for i in range(length):
    DR_i = str(Rating[i])
    Item_i = ItemID[i]
    for j in range(i + 1, length):
        print('%s:%s\t%s:%s' % (Item_i, ItemID[j], DR_i, str(Rating[j])))


In [ ]:
import sys
import numpy as np


line = sys.stdin.readline().rstrip('\n')
key, value = line.split('\t')
Item_I, Item_J = key.split(':')
DR_i, DR_j = value.split(':')
DRA = np.zeros(1) + float(DR_i)
DRB = np.zeros(1) + float(DR_j)

for line in sys.stdin:
    line = line.strip()
    key, value = line.split('\t')
    Item_i, Item_j = key.split(':')
    DR_i, DR_j = value.split(':')
    if Item_i == Item_I and Item_j == Item_J:
        DRA = np.append(DRA, float(DR_i))
        DRB = np.append(DRB, float(DR_j))
    else:
        nom = np.sum(DRA * DRB)
        denom = np.sum(DRA ** 2) * np.sum(DRB ** 2)
        if denom != 0:
            nom /= denom
        else:
            nom = 0
        if nom < 0:
            nom = 0
        print("(%s:%s)\t%s" % (Item_I, Item_J, nom))
        Item_I, Item_J = Item_i, Item_j
        DRA = np.zeros(1) + float(DR_i)
        DRB = np.zeros(1) + float(DR_j)
nom = np.sum(DRA * DRB)
denom = np.sum(DRA ** 2) * np.sum(DRB ** 2)
if denom != 0:
    nom /= denom
else:
    nom = 0
if nom < 0:
    nom = 0
print("%s:%s\t%s" % (Item_I, Item_J, nom))

In [ ]:
import sys


for line in sys.stdin:
    line = line.strip()
    UserID, key, R = line.split(',')
    value = 'A:' + UserID + ':' + R
    print('%s\t%s' % (key, value))


In [ ]:
import sys


for line in sys.stdin:
    key, value = line.strip().split('\t')
    key_1, key_2 = key.strip().split(':')
    value_1 = 'B:' + key_2 + value
    value_2 = 'B:' + key_1 + value
    print('%s\t%s' % (key_1, value_1))
    print('%s\t%s' % (key_2, value_2))


In [ ]:
# strange reducer

import sys
import numpy as np


line = sys.stdin.readline().rstrip('\n')
key, value = line.split('\t')
Item_I, Item_J = key.split(':')
DR_i, DR_j = value.split(':')
DRA = np.zeros(1) + float(DR_i)
DRB = np.zeros(1) + float(DR_j)

for line in sys.stdin:
    line = line.strip()
    key, value = line.split('\t')
    Item_i, Item_j = key.split(':')
    DR_i, DR_j = value.split(':')
    if Item_i == Item_I and Item_j == Item_J:
        DRA = np.append(DRA, float(DR_i))
        DRB = np.append(DRB, float(DR_j))
    else:
        nom = np.sum(DRA * DRB)
        denom = np.sum(DRA ** 2) * np.sum(DRB ** 2)
        if denom != 0:
            nom /= denom
        else:
            nom = 0
        if nom < 0:
            nom = 0
        print("(%s:%s)\t%s" % (Item_I, Item_J, nom))
        Item_I, Item_J = Item_i, Item_j
        DRA = np.zeros(1) + float(DR_i)
        DRB = np.zeros(1) + float(DR_j)
nom = np.sum(DRA * DRB)
denom = np.sum(DRA ** 2) * np.sum(DRB ** 2)
if denom != 0:
    nom /= denom
else:
    nom = 0
if nom < 0:
    nom = 0
print("%s:%s\t%s" % (Item_I, Item_J, nom))

In [ ]:
import sys


for line in sys.stdin:
    print(line)


In [ ]:
import sys

line = sys.stdin.readline().rstrip('\n')
key, value = line.split('\t')
Rating, S = value.split(':')


for line in sys.stdin:
    key, value = line.strip().split('\t')
    key_1, key_2 = key.strip().split(':')
    value_1 = 'B:' + key_2 + value
    value_2 = 'B:' + key_1 + value
    print('%s\t%s' % (key_1, value_1))
    print('%s\t%s' % (key_2, value_2))